### Check if configuration has been run already and change directory to repository root:

In [1]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


In [2]:
import datajoint as dj
import pandas as pd

import u19_pipeline.ephys_pipeline as ephys_pipeline
from u19_pipeline.ephys_pipeline import  ephys_element

import u19_pipeline.imaging_pipeline as imaging_pipeline
from u19_pipeline.imaging_pipeline import  imaging_element, scan_element

import u19_pipeline.automatic_job.recording_handler as rec_handler
import u19_pipeline.automatic_job.recording_process_handler as rec_process_handler

from u19_pipeline import recording, recording_process


Connecting alvaros@datajoint00.pni.princeton.edu:3306


### Run recording main pipeline

In [3]:
l = rec_handler.RecordingHandler.pipeline_handler_main()

status_recording_id > -1 and status_recording_id < 3
*recording_id  *location      recording_moda status_recordi task_copy_id_p recording_dire local_director ip_address     system_user    acquisition_ty
+------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+
11             C42-2photonMic imaging        2              None           hnieh/hnieh_E8 /Users/alvaros None           None           2photon       
12             C42-Mesoscope  imaging        2              None           emdia/emdia_gp /Users/alvaros None           None           mesoscope     
 (Total: 2)

function to apply: modality_preingestion
rec_series['query_key']
{'recording_id': 11}


ScanInfo: 0it [00:00, ?it/s]


recording_key_dict {'recording_id': 11}
status 3
update_field None
update_value None
update_status_dict {'recording_id': 11, 'status_recording_id': 3}
error_info_dict {'error_message': None, 'error_exception': None}
function to apply: modality_preingestion
rec_series['query_key']
{'recording_id': 12}


ScanInfo:   0%|          | 0/3 [00:00<?, ?it/s]

get_scan_image_files  .........
rec_process_key {'recording_id': 12, 'tiff_split': 1, 'scan_id': 0} scan_key {'recording_id': 12}


ScanInfo:   0%|          | 0/3 [35:28<?, ?it/s]


KeyboardInterrupt: 

### Run recording_prcoess main pipeline

In [3]:
l = rec_process_handler.RecProcessHandler.pipeline_handler_main()

   recording_id  job_id recording_modality  status_processing_id  \
0            11     208            imaging                     2   

   fragment_number recording_process_pre_path  \
0                1   hnieh/hnieh_E84/20190528   

           recording_process_post_path                      task_copy_id_pre  \
0  hnieh/hnieh_E84/20190528/job_id_208  10a00bfe-0de9-11ed-8dae-9f359c660fbd   

  task_copy_id_post slurm_id  
0              None     None  
{'process_cluster': 'tiger', 'process_repository': 'BrainCogsEphysSorters', 'process_script': 'main_script.py'}
scp /Users/alvaros/Documents/MATLAB/BrainCogsProjects/Datajoint_proj/U19-pipeline_python/u19_pipeline/automatic_job/ParameterFiles/process_paramset_208.json alvaros@tigergpu.princeton.edu:/scratch/gpfs/BRAINCOGS/ParameterFiles/process_paramset_208.json
0
scp /Users/alvaros/Documents/MATLAB/BrainCogsProjects/Datajoint_proj/U19-pipeline_python/u19_pipeline/automatic_job/ParameterFiles/preprocess_paramset_208.json alvaros@tigerg

In [16]:
from u19_pipeline import utility

rec_process_keys = dict()
rec_process_keys['sdsd'] = 207


params_df = pd.DataFrame((imaging_pipeline.imaging_element.ProcessingParamSet.proj('params', 'processing_method') * \
recording_process.Processing.ImagingParams.proj('paramset_idx') & rec_process_keys).fetch(as_dict=True))
params_df = params_df.drop('paramset_idx', axis=1)

#Insert processing_method in params itself (for BrainCogsImagingSorters)
params_df['params'] = params_df.apply(lambda x: {**x['params'], **{'processing_method':x['processing_method']}},axis=1)

# Get preprocess param sets
preparams_df = pd.DataFrame((imaging_pipeline.imaging_element.PreProcessParamSteps * \
utility.smart_dj_join(imaging_pipeline.imaging_element.PreProcessParamSteps.Step, imaging_pipeline.imaging_element.PreProcessParamSet.proj('preprocess_method', 'params')) *
recording_process.Processing.ImagingParams.proj('preprocess_param_steps_id') & rec_process_keys).fetch(as_dict=True))

if preparams_df.shape[0] > 0:

       # Join precluster params for the same recording_process
       preparams_df['preparams'] = preparams_df.apply(lambda x : {x['preprocess_method']: x['params']}, axis=1)
       preparams_df = preparams_df.sort_values(by=['job_id', 'step_number'])
       preparams_df = preparams_df[['job_id', 'preparams']].groupby("job_id").agg(lambda x: list(x))
       preparams_df = preparams_df.reset_index()

else:
       preparams_df = pd.DataFrame((imaging_pipeline.imaging_element.PreProcessParamSteps * \
              recording_process.Processing.ImagingParams.proj('preprocess_param_steps_id') & rec_process_keys).fetch(as_dict=True))
       preparams_df['preparams'] = None

params_df = params_df.merge(preparams_df)

print('params_df 3')
print(params_df)

params_df 3
   job_id processing_method  \
0     207           suite2p   
1     208           suite2p   
2     209           suite2p   
3     210           suite2p   
4     211           suite2p   

                                              params  \
0  {'look_one_level_down': 0.0, 'fast_disk': [], ...   
1  {'look_one_level_down': 0.0, 'fast_disk': [], ...   
2  {'look_one_level_down': 0.0, 'fast_disk': [], ...   
3  {'look_one_level_down': 0.0, 'fast_disk': [], ...   
4  {'look_one_level_down': 0.0, 'fast_disk': [], ...   

   preprocess_param_steps_id preprocess_param_steps_name  \
0                          0            No preprocessing   
1                          0            No preprocessing   
2                          0            No preprocessing   
3                          0            No preprocessing   
4                          0            No preprocessing   

  preprocess_param_steps_desc preparams  
0      No preprocessing steps      None  
1      No preproces

### Insert cat_gt params 1 & 2

In [3]:
cat_gt_params =  {
      "apfilter": ["biquad",2,300,0],
      "gfix": [0.40,0.10,0.02],
      "extras": ["prb_fld", "t_miss_ok", "ap", "gblcar", "out_prb_fld"],
      "lazy": True
   }
 
ephys_element.PreClusterParamSet.insert_new_params(
    precluster_method='catgt',
    paramset_idx=2,
    params=cat_gt_params,
    paramset_desc='Spike sorting using Kilosort w lazy')


cat_gt_params =  {
      "apfilter": ["biquad",2,301,0],
      "gfix": [0.40,0.10,0.02],
      "extras": ["prb_fld", "t_miss_ok", "ap", "gblcar", "out_prb_fld"],
      "lazy": True
   }
 
ephys_element.PreClusterParamSet.insert_new_params(
    precluster_method='catgt2',
    paramset_idx=3,
    params=cat_gt_params,
    paramset_desc='Spike sorting using Kilosort w lazy')


ephys_element.PreClusterParamSet()
 


paramset_idx,precluster_method,paramset_desc,param_set_hash,params dictionary of all applicable parameters
0,catgt,Catgt base params,4271fe75-1732-5821-0070-45b4c286e867,=BLOB=
1,catgt2,Catgt test params,3d1047b9-deec-bc7f-d5a5-4e30c1665f07,=BLOB=
2,catgt,Catgt base params w lazy,724eb89a-e0ac-00b9-db32-0b93822caf7b,=BLOB=
3,catgt2,Catgt test params w lazy,ac733d65-f3cc-1c60-e464-1adcbc4887ec,=BLOB=


### Insert precluster_param_lists

In [5]:
paramlist = {'precluster_param_steps_id':0}
ephys_element.PreClusterParamSteps.insert1(paramlist, skip_duplicates=True)

paramorder = {
'precluster_param_steps_id': 0,
'step_number': 1,
'paramset_idx': 0
}
ephys_element.PreClusterParamSteps.Step.insert1(paramorder, skip_duplicates=True)


paramlist = {'precluster_param_steps_id':1}
ephys_element.PreClusterParamSteps.insert1(paramlist, skip_duplicates=True)

paramorder = {
'precluster_param_steps_id': 1,
'step_number': 1,
'paramset_idx': 1
}
ephys_element.PreClusterParamSteps.Step.insert1(paramorder, skip_duplicates=True)


paramlist = {'precluster_param_steps_id':2}
ephys_element.PreClusterParamSteps.insert1(paramlist,skip_duplicates=True)

paramorder = {
'precluster_param_steps_id': 2,
'step_number': 1,
'paramset_idx': 0
}
ephys_element.PreClusterParamSteps.Step.insert1(paramorder, skip_duplicates=True)
paramorder = {
'precluster_param_steps_id': 2,
'step_number': 2,
'paramset_idx': 1
}
ephys_element.PreClusterParamSteps.Step.insert1(paramorder, skip_duplicates=True)

ephys_element.PreClusterParamSteps.Step()


precluster_param_steps_id,step_number Order of operations,paramset_idx
0,1,0
2,1,0
1,1,1
2,2,1


### Insert cluster_paramsets

In [16]:
params = {
 "fs": 30000,
 "fshigh": 150,
 "minfr_goodchannels": 0.1,
 "Th": [10, 4],
 "lam": 10,
 "AUCsplit": 0.9,
 "minFR": 0.02,
 "momentum": [20, 400],
 "sigmaMask": 30,
 "ThPre": 8,
 "CAR": 1, 
 "spkTh": -6,
 "reorder": 1,
 "nskip": 25,
 "GPU": 1,
 "Nfilt": 1024,
 "nfilt_factor": 4,
 "ntbuff": 64,
 "NT": 32832,
 "whiteningRange": 32,
 "nSkipCov": 25,
 "scaleproc": 200,
 "nPCs": 3,
 "useRAM": 0,
 "trange": [0, 1000000000],
"NchanTOT": 384
}

ephys_element.ClusteringParamSet.insert_new_params(
    processing_method='kilosort2',
    paramset_idx=0,
    params=params,
    paramset_desc='Spike sorting using Kilosort2')


params = {
 "fs": 30000,
 "fshigh": 150,
 "minfr_goodchannels": 0.1,
 "Th": [10, 4],
 "lam": 10,
 "AUCsplit": 0.9,
 "minFR": 0.02,
 "momentum": [20, 400],
 "sigmaMask": 30,
 "ThPre": 8,
 "CAR": 1, 
 "spkTh": -6,
 "reorder": 1,
 "nskip": 25,
 "GPU": 2,
 "Nfilt": 1024,
 "nfilt_factor": 4,
 "ntbuff": 64,
 "NT": 32832,
 "whiteningRange": 32,
 "nSkipCov": 25,
 "scaleproc": 200,
 "nPCs": 3,
 "useRAM": 0,
 "trange": [0, 1000000000],
"NchanTOT": 384
}

ephys_element.ClusteringParamSet.insert_new_params(
    processing_method='kilosort2',
    paramset_idx=1,
    params=params,
    paramset_desc='Spike sorting using Kilosort2')


paramset_idx,clustering_method,paramset_desc,param_set_hash,params dictionary of all applicable parameters
0,kilosort2,Spike sorting using Kilosort2,b9e07e55-95ea-3463-a740-90291de41da6,=BLOB=
1,kilosort2,Spike sorting using Kilosort2,697d24ac-2d46-a6e8-6f6b-2d1afaea000d,=BLOB=
2,kilosort,Spike sorting using Kilosort2,e6bfd993-55fc-5844-bdfe-30fab9888f28,=BLOB=


### Insert Manuel config params for Kilosort 3

In [17]:

ops = dict()
# sample rate
ops['fs'] = 30000;  
ops['fshigh'] = 300;   

# minimum firing rate on a "good" channel (0 to skip)
ops['minfr_goodchannels'] = 0; 

# threshold on projections (like in Kilosort1, can be different for last pass like [10 4])
ops['Th'] = [10, 4];  

# how important is the amplitude penalty (like in Kilosort1, 0 means not used, 10 is average, 50 is a lot) 
ops['lam'] = 10;  

# splitting a cluster at the end requires at least this much isolation for each sub-cluster (max = 1)
ops['AUCsplit'] = 0.9; 

# minimum spike rate (Hz), if a cluster falls below this for too long it gets removed
ops['minFR'] = 1/50; 

# number of samples to average over (annealed from first to second value) 
ops['momentum'] = [20, 400]; 

# spatial constant in um for computing residual variance of spike
ops['sigmaMask'] = 30; 

# threshold crossings for pre-clustering (in PCA projection space)
ops['ThPre'] = 8; 
ops['reorder']         = 1;       # whether to reorder batches for drift correction. 
ops['nskip']           = 25;      # how many batches to skip for determining spike PCs

# danger, changing these settings can lead to fatal errors
# options for determining PCs
ops['spkTh']           = -6;      # spike threshold in standard deviations (-6)

ops['GPU']                 = 1; # has to be 1, no CPU version yet, sorry
# ops.Nfilt               = 1024; # max number of clusters
ops['nfilt_factor']        = 4; # max number of clusters per good channel (even temporary ones)
ops['ntbuff']              = 64;    # samples of symmetrical buffer for whitening and spike detection
ops['NT']                  = 64*1024+ ops['ntbuff']; # must be multiple of 32 + ntbuff. This is the batch size (try decreasing if out of memory). 
ops['whiteningRange']      = 32; # number of channels to use for whitening each channel
ops['nSkipCov']            = 25; # compute whitening matrix from every N-th batch
ops['scaleproc']           = 200;   # int16 scaling of whitened data
ops['nPCs']                = 3; # how many PCs to project the spikes into
ops['useRAM']              = 0; # not yet available


ops['trange']              = [0, 100000000000000]
ops['NchanTOT']            = 384
ops['sig']                 = 20;  #spatial smoothness constant for registration
ops['nblocks']             = 5; # blocks for registration. 0 turns it off, 1 does rigid registration. Replaces "datashift" option. 

ephys_element.ClusteringParamSet.insert_new_params(
    processing_method='kilosort',
    paramset_idx=2,
    params=ops,
    paramset_desc='Spike sorting using Kilosort2')


ephys_element.ClusteringParamSet()

/Users/alvaros/opt/anaconda3/envs/u19_datajoint/lib/python3.7/site-packages/datajoint/connection.py:304: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


paramset_idx,clustering_method,paramset_desc,param_set_hash,params dictionary of all applicable parameters
0,kilosort2,Spike sorting using Kilosort2,b9e07e55-95ea-3463-a740-90291de41da6,=BLOB=
1,kilosort2,Spike sorting using Kilosort2,697d24ac-2d46-a6e8-6f6b-2d1afaea000d,=BLOB=
2,kilosort,Spike sorting using Kilosort2,e6bfd993-55fc-5844-bdfe-30fab9888f28,=BLOB=


### Insert Suite 2p default params 

In [3]:
ops = {
     "look_one_level_down": 0.0,
     "fast_disk": [],
     "delete_bin": False,
     "mesoscan": False,
     "h5py": [],
     "h5py_key": "data",
     "save_path0": [],
     "subfolders": [],
     "nplanes": 1,
     "nchannels": 1,
     "functional_chan": 1,
     "tau": 1.0,
     "fs": 10.0,
     "force_sktiff": False,
     "preclassify": 0.0,
     "save_mat": False,
     "combined": True,
     "aspect": 1.0,
     "do_bidiphase": False,
     "bidiphase": 0.0,
     "do_registration": True,
     "keep_movie_raw": False,
     "nimg_init": 300,
     "batch_size": 500,
     "maxregshift": 0.1,
     "align_by_chan": 1,
     "reg_tif": False,
     "reg_tif_chan2": False,
     "subpixel": 10,
     "smooth_sigma": 1.15,
     "th_badframes": 1.0,
     "pad_fft": False,
     "nonrigid": False,
     "block_size": [128, 128],
     "snr_thresh": 1.2,
     "maxregshiftNR": 5.0,
     "1Preg": False,
     "spatial_hp": 50.0,
     "pre_smooth": 2.0,
     "spatial_taper": 50.0,
     "roidetect": True,
     "sparse_mode": False,
     "diameter": 12,
     "spatial_scale": 0,
     "connected": True,
     "nbinned": 5000,
     "max_iterations": 20,
     "threshold_scaling": 1.0,
     "max_overlap": 0.75,
     "high_pass": 100.0,
     "inner_neuropil_radius": 2,
     "min_neuropil_pixels": 350,
     "allow_overlap": False,
     "chan2_thres": 0.65,
     "baseline": "maximin",
     "win_baseline": 60.0,
     "sig_baseline": 10.0,
     "prctile_baseline": 8.0,
     "neucoeff": 0.7,
     "xrange": [0, 0],
     "yrange": [0, 0]
}


imaging_element.ProcessingParamSet.insert_new_params(
    processing_method='suite2p',
    paramset_idx=0,
    params=ops,
    paramset_desc='Default params Suite2p')

### Insert default Imaging PreProcess (no preprocess)

In [6]:
PreProcessParamStepsRecord = dict()
PreProcessParamStepsRecord['preprocess_param_steps_id'] = 0 
PreProcessParamStepsRecord['preprocess_param_steps_name'] = 'No preprocessing'
PreProcessParamStepsRecord['preprocess_param_steps_desc'] = 'No preprocessing steps'

imaging_element.PreProcessParamSteps.insert1(PreProcessParamStepsRecord, skip_duplicates=True)

### Insert defaultParmas table for some recordings

In [14]:

# For recording 39, all probes equal, list = 0
default_params_key = dict()
default_params_key['recording_id'] = 39
default_params_key['fragment_number'] = 0
default_params_key['preprocess_param_steps_id'] = 0
default_params_key['paramset_idx'] = 0


recording.DefaultParams.insert1(default_params_key, skip_duplicates=True)

# For recording 41, all probes equal, list = 2   (list 2 has two different preparams catgt & catgt2)
default_params_key = dict()
default_params_key['recording_id'] = 41
default_params_key['fragment_number'] = 0
default_params_key['preprocess_param_steps_id'] = 2
default_params_key['paramset_idx'] = 0


recording.DefaultParams.insert1(default_params_key, skip_duplicates=True)

# For recording 40, all probes different params
default_params_key = dict()
default_params_key['recording_id'] = 40
default_params_key['fragment_number'] = 0
default_params_key['preprocess_param_steps_id'] = 0
default_params_key['paramset_idx'] = 0
default_params_key['default_same_preparams_all'] = 0
default_params_key['default_same_params_all'] = 0

recording.DefaultParams.insert1(default_params_key, skip_duplicates=True)

default_params_key = dict()
default_params_key['recording_id'] = 40
default_params_key['fragment_number'] = 1
default_params_key['preprocess_param_steps_id'] = 0
default_params_key['paramset_idx'] = 1
default_params_key['default_same_preparams_all'] = 0
default_params_key['default_same_params_all'] = 0

recording.DefaultParams.insert1(default_params_key, skip_duplicates=True)

default_params_key = dict()
default_params_key['recording_id'] = 40
default_params_key['fragment_number'] = 2
default_params_key['preprocess_param_steps_id'] = 1
default_params_key['paramset_idx'] = 0
default_params_key['default_same_preparams_all'] = 0
default_params_key['default_same_params_all'] = 0

recording.DefaultParams.insert1(default_params_key, skip_duplicates=True)

default_params_key = dict()
default_params_key['recording_id'] = 40
default_params_key['fragment_number'] = 3
default_params_key['preprocess_param_steps_id'] = 1
default_params_key['paramset_idx'] = 1
default_params_key['default_same_preparams_all'] = 0
default_params_key['default_same_params_all'] = 0


recording.DefaultParams.insert1(default_params_key, skip_duplicates=True)

recording.DefaultParams()


recording_id Unique number assigned to recording,fragment_number probe/field_of_view # if not always the same,default_same_preparams_all by default all probes/fields of view have same preparameters,pre_param_list_id preparams index for recording (could be imaging/ephys),default_same_params_all by default all probes/fields of view have same parameters,paramset_idx params index for recording (could be imaging/ephys)
37,0,1,0,1,0
39,0,1,0,1,0
40,0,0,0,0,0
40,1,0,0,0,1
40,2,0,1,0,0
40,3,0,1,0,1
41,0,1,2,1,0


## "Simplified" Ephys diagram and Imaging diagram

In [3]:
ephys_pipeline = dj.create_virtual_module('ephys_pipeline', 'u19_ephys_pipeline')
ephys_element = dj.create_virtual_module('ephys_element', 'u19_pipeline_ephys_element')
probe_element = dj.create_virtual_module('probe_element', 'u19_pipeline_probe_element')


imaging_pipeline = dj.create_virtual_module('imaging_pipeline', 'u19_imaging_pipeline')
scan_element = dj.create_virtual_module('scan_element', 'u19_pipeline_scan_element')
imaging_element = dj.create_virtual_module('imaging_element', 'u19_pipeline_imaging_element')


In [4]:
from u19_pipeline.ephys_pipeline import get_spikeglx_meta_filepath, get_session_directory

so = recording.Recording.fetch("KEY", as_dict=True)



print(so[-1])
print(get_session_directory(so[-1]))
get_spikeglx_meta_filepath(so[-1])


NameError: name 'recording' is not defined

In [5]:
dj.ERD(recording) + dj.ERD(recording_process) + dj.ERD(ephys_pipeline)  + dj.ERD(ephys_element.EphysRecording) + dj.ERD(ephys_element.PreClusterTask) + dj.ERD(ephys_element.PreClusterParamList)


NameError: name 'recording' is not defined

In [6]:
dj.ERD(recording) + dj.ERD(recording_process) + dj.ERD(imaging_pipeline) + dj.ERD(scan_element.Scan) + dj.ERD(imaging_element.ProcessingTask) + dj.ERD(imaging_element.ProcessingParamSet)


NameError: name 'recording' is not defined

# DANGER, delete EphysPipeline tables

In [6]:
#ephys_pipeline_db = dj.create_virtual_module('ephys_pipeline', 'u19_ephys_pipeline')
#ephys_pipeline_db.EphysPipelineSession.delete()

,job_id,recording_id,status_processing_id,fragment_number,recording_process_pre_path,recording_process_post_path,task_copy_id_pre,task_copy_id_post,slurm_id
0,56,39,-1,0,ms81/ms81_M011/20211116/TowersTask_g0/TowersTa...,None,b5073818-daad-4a46-af50-6ec1519de31f,None,None
1,57,39,-1,1,ms81/ms81_M011/20211116/TowersTask_g0/TowersTa...,None,None,None,None
2,58,39,-1,2,ms81/ms81_M011/20211116/TowersTask_g0/TowersTa...,None,None,None,None
3,59,40,-1,0,ms81/ms81_M011/20211120/TowersTask_g0/TowersTa...,None,None,None,None
4,60,40,-1,1,ms81/ms81_M011/20211120/TowersTask_g0/TowersTa...,None,None,None,None
5,61,40,-1,2,ms81/ms81_M011/20211120/TowersTask_g0/TowersTa...,None,None,None,None
6,62,40,-1,3,ms81/ms81_M011/20211120/TowersTask_g0/TowersTa...,None,None,None,None
7,63,41,-1,0,ms81/ms81_M012/20211210/2021-12-10_g0/2021-12-...,None,None,None,None
8,64,41,-1,1,ms81/ms81_M012/20211210/2021-12-10_g0/2021-12-...,None,None,None,None
9,65,41,-1,2,ms81/ms81_M012/20211210/2021-12-10_g0/2021-12-...,None,None,None,None
